In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam


In [2]:
datatexts = pd.read_csv("data1.csv", delimiter=';', skiprows=0, low_memory=False)
datatexts.columns=['txt', 'tag', 'fake']
datatexts.head()

,txt,tag,fake
0,4 октября 2001 года в нейтральных водах аквато...,самолёт,0
1,45 минут назад возле международного аэропорта ...,самолёт,1
2,Польша находится на грани отправки военных сам...,самолёт,1
3,В России стали массово «минировать» самолеты. ...,самолёт,1
4,"Международный аэропорт Мале Велана (MLE), глав...",самолёт,0


In [3]:
def preprocess_text(text):
    
    #удаляем символы пунктуации и числа
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    
    text = text.lower()
    
    #удаляем спецсимволы
    text = re.sub(r'[^\w\s\d]', '', text)
    
    #разбиваем на токенф
    tokens = nltk.word_tokenize(text)
    
    #удаляем стопслова
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]
    
    #лемматизируем токены
    morph = MorphAnalyzer()
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [4]:
stopwords.words('russian')

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [5]:
for i in range(len(datatexts['txt'])):
    datatexts['txt'][i] = preprocess_text(datatexts['txt'][i]) 

datatexts.to_csv("clean_datatexts.csv")

C:\Users\milp00ps\AppData\Local\Temp\ipykernel_14712\2934256406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatexts['txt'][i] = preprocess_text(datatexts['txt'][i])


In [6]:
datatexts

,txt,tag,fake
0,октябрь год нейтральный вода акватория чёрный ...,самолёт,0
1,минута назад возле международный аэропорт шере...,самолёт,1
2,польша находиться грань отправка военный самол...,самолёт,1
3,россия стать массово минировать самолёт нескол...,самолёт,1
4,международный аэропорт мале велана mle главный...,самолёт,0
5,гонконгский лоукостер hk express объявить приб...,самолёт,0
6,российский правительственный корабль вызвать т...,самолёт,0
7,мужчина предстать датский суд сесть самолёт па...,самолёт,1
8,летноиспытательный самолёт airbus axlr вернуть...,самолёт,0
9,россия создать уникальный самолёт способный ле...,самолёт,1


In [7]:
datatexts1 = shuffle(shuffle(shuffle(datatexts)))

y_ = datatexts['tag']

list_labels = np.array(list(map(lambda x:
                               0 if x == 'самолёт' else
                               1 if x == 'животные' else 2, y_)))

X_train, X_test, y_train, y_test = train_test_split(datatexts['txt'], list_labels, test_size=0.3, random_state=42)

In [8]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_counts = vectorizer.transform(X_train)
X_train_matrix = vectorizer.fit_transform(X_train).toarray()
X_test_matrix = vectorizer.transform(X_test).toarray()

In [9]:
np.savetxt("matrix.csv", X_train_matrix, delimiter=",")

In [10]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_matrix.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

opt = Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_matrix, y_train, epochs=10, batch_size=20, validation_data=(X_test_matrix, y_test))

loss, accuracy = model.evaluate(X_test_matrix, y_test, batch_size=20)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/10
2/2 [==============================] - 2s 385ms/step - loss: 1.4350 - accuracy: 0.4286 - val_loss: 1.0379 - val_accuracy: 0.6667
Epoch 2/10
2/2 [==============================] - 0s 71ms/step - loss: 1.0596 - accuracy: 0.5714 - val_loss: 1.0069 - val_accuracy: 0.6667
Epoch 3/10
2/2 [==============================] - 0s 66ms/step - loss: 0.6964 - accuracy: 0.7619 - val_loss: 0.9752 - val_accuracy: 0.6667
Epoch 4/10
2/2 [==============================] - 0s 66ms/step - loss: 0.6504 - accuracy: 0.7143 - val_loss: 0.9374 - val_accuracy: 0.6667
Epoch 5/10
2/2 [==============================] - 0s 65ms/step - loss: 0.4806 - accuracy: 0.8571 - val_loss: 0.8848 - val_accuracy: 0.6667
Epoch 6/10
2/2 [==============================] - 0s 65ms/step - loss: 0.5196 - accuracy: 0.6667 - val_loss: 0.8422 - val_accuracy: 0.6667
Epoch 7/10
2/2 [==============================] - 0s 70ms/step - loss: 0.4454 - accuracy: 0.8095 - val_loss: 0.7951 - val_accuracy: 0.6667
Epoch 8/10
2/2 [==========

In [11]:
labels = ['Самолеты', 'Животные', 'Музыка']

#получаем вероятность принадлежности к какому-либо классу
probabilities = model.predict(X_test_matrix)

predictions = np.argmax(probabilities, axis=1)

text_predictions = [labels[p] for p in predictions]
print(text_predictions)
print(X_test.values)

1/1 [==============================] - 0s 200ms/step
['Животные', 'Животные', 'Музыка', 'Животные', 'Самолеты', 'Самолеты', 'Животные', 'Музыка', 'Животные']
['церемония вручение премия grammy год артистка beyoncé стать сам награждать женщина история премия удостоиться награда'
 'австралия обнаружить новый вид голубеть чёрный голубь необычный чёрный окраска отличие традиционный синесерый цвет вид назвать вороний голубь изз свой тёмный окраска'
 'год группа acdc выпустить свой новый альбом power up который стать первый релиз группа смерть гитарист малкольм янг год'
 'намибия объявить открытие новый национальный парк площадь тысяча квадратный километр стать дом многий вид животное число крупный популяция африканский слон планета'
 'летноиспытательный самолёт airbus axlr вернуться четырёхдневный наземный испытание холодный погода икалуит север канада цель поездка подтверждение работа самолёт холодный погода наземный операция несколько неделя ранее другой испытательный самолёт axlr соверши

In [12]:
X2_test = X_test
X2_train = X_train
Y2_train = np.array([], int)
for i in range(len(datatexts['txt'])):
    if datatexts['txt'][i] in X2_train.values:
        Y2_train = np.append(Y2_train, datatexts['fake'][i])
Y2_test = np.array([], int)
for i in range(len(datatexts)):
    if datatexts['txt'][i] in X2_test.values:
        Y2_test = np.append(Y2_test, datatexts['fake'][i])


In [13]:
vectorizer2 = CountVectorizer()
vectorizer2.fit(X2_train)
X2_train_counts = vectorizer2.transform(X2_train)
X2_train_matrix = vectorizer2.fit_transform(X2_train).toarray()
X2_test_matrix = vectorizer2.transform(X2_test).toarray()

In [29]:
model2 = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X2_train_matrix.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

opt2 = Adam(learning_rate=0.001)
model2.compile(optimizer=opt2,
              loss='binary_crossentropy',
              metrics=['accuracy'])


model2.fit(X2_train_matrix, Y2_train, epochs=15, batch_size=4, validation_split=0.35)


loss, accuracy = model2.evaluate(X2_test_matrix, Y2_test, batch_size=4)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/15
4/4 [==============================] - 1s 81ms/step - loss: 0.8025 - accuracy: 0.3077 - val_loss: 0.6944 - val_accuracy: 0.5000
Epoch 2/15
4/4 [==============================] - 0s 27ms/step - loss: 0.7007 - accuracy: 0.5385 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 3/15
4/4 [==============================] - 0s 27ms/step - loss: 0.7695 - accuracy: 0.6154 - val_loss: 0.6954 - val_accuracy: 0.3750
Epoch 4/15
4/4 [==============================] - 0s 24ms/step - loss: 0.7173 - accuracy: 0.4615 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch 5/15
4/4 [==============================] - 0s 24ms/step - loss: 0.8343 - accuracy: 0.4615 - val_loss: 0.6969 - val_accuracy: 0.3750
Epoch 6/15
4/4 [==============================] - 0s 23ms/step - loss: 0.6653 - accuracy: 0.6154 - val_loss: 0.6968 - val_accuracy: 0.3750
Epoch 7/15
4/4 [==============================] - 0s 25ms/step - loss: 0.7700 - accuracy: 0.5385 - val_loss: 0.6962 - val_accuracy: 0.3750
Epoch 8/15
4/4 [===========

In [30]:
fake = [1, 0]

#получаем вероятность принадлежности к какому-либо классу
probs = model2.predict(X2_test_matrix)

preds = np.argmax(probs, axis=1)

fake_predictions = [fake[p] for p in preds]
print(fake_predictions)
print(X2_test)

1/1 [==============================] - 0s 76ms/step
[1, 1, 1, 1, 1, 1, 1, 1, 1]
27    церемония вручение премия grammy год артистка ...
15    австралия обнаружить новый вид голубеть чёрный...
23    год группа acdc выпустить свой новый альбом po...
17    намибия объявить открытие новый национальный п...
8     летноиспытательный самолёт airbus axlr вернуть...
9     россия создать уникальный самолёт способный ле...
28    певецмультиинструменталист майк олдфилд выпуст...
24    топовый рэпер eminem объявить свой уход музыка...
12    научный коллектив норвегия сообщить создание п...
Name: txt, dtype: object
